# **INFERENCE**
___
___

## **INITIALIZATION**

### *IMPORTS*

In [20]:
from IPython.display import display

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

### *DATA LOADING*

In [25]:
train_df = pd.read_csv("train.csv", index_col=0, names=["val1", "val2", "val3", "val4"])
train_ds = torch.from_numpy(train_df.values)

In [26]:
display(train_df)

,val1,val2,val3,val4
0,0.012495,0.011126,0.003252,0.006625
1,0.011439,0.002691,0.001206,0.006947
2,0.000632,0.007277,0.004049,0.000074
3,0.017828,0.028210,0.007758,0.007382
4,0.021115,0.019642,0.009238,0.011499
...,...,...,...,...
741,0.001938,0.008833,0.003927,0.005106
742,0.005003,0.018943,0.003057,0.001988
743,0.007683,0.001958,0.007002,0.006467
744,0.003396,0.001280,0.007621,0.001680


In [28]:
train_ds

tensor([[1.2495e-02, 1.1126e-02, 3.2520e-03, 6.6249e-03],
        [1.1439e-02, 2.6906e-03, 1.2064e-03, 6.9467e-03],
        [6.3166e-04, 7.2774e-03, 4.0487e-03, 7.4499e-05],
        ...,
        [7.6830e-03, 1.9576e-03, 7.0022e-03, 6.4668e-03],
        [3.3956e-03, 1.2796e-03, 7.6207e-03, 1.6801e-03],
        [4.5912e-03, 6.6749e-03, 7.6000e-03, 9.5599e-03]], dtype=torch.float64)

___

## **DATA PREPROCESSING**